# A movie of sea-surface velocities with cartopy

In [1]:
import xarray as xr
import numpy as np
import time
import cftime
import matplotlib.pyplot as plt
import cosima_cookbook as cc
import matplotlib.path as mpath
import matplotlib.patheffects as PathEffects
from matplotlib import ticker
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
import cartopy.feature as cfeature
from matplotlib import gridspec
import os
from pathlib import Path

import warnings
warnings.filterwarnings('ignore') # suppress warnings

from dask.distributed import Client

In [2]:
client = Client()        
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 8
Total threads: 48,Total memory: 188.55 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34441,Workers: 8
Dashboard: /proxy/8787/status,Total threads: 48
Started: Just now,Total memory: 188.55 GiB
Comm: tcp://127.0.0.1:36601,Total threads: 6
Dashboard: /proxy/36835/status,Memory: 23.57 GiB
Nanny: tcp://127.0.0.1:34765,


2024-07-19 22:13:02,960 - distributed.scheduler - ERROR - Task 'getattr-519bb2b8-96ea-4c1d-a278-70058e201647' has 81.47 GiB worth of input dependencies, but worker tcp://127.0.0.1:41471 has memory_limit set to 23.57 GiB.
2024-07-19 22:13:02,962 - distributed.scheduler - ERROR - Task '_preprocess-37faa17d-ec1d-457d-9d1c-9d03422df321' has 81.47 GiB worth of input dependencies, but worker tcp://127.0.0.1:41471 has memory_limit set to 23.57 GiB.


### Some options for the animation

In [3]:
frame_rate = 30     # frames-per-second
resolution = 1080

movie_name = 'test.mp4'

frame_directory = './movie_frames/'
print("movie frames will be saved in:", frame_directory)

movie frames will be saved in: ./movie_frames/


If the directory to save movie frames doesn't exist we create it.

In [4]:
Path(frame_directory).mkdir(parents=True, exist_ok=True)

### Load in daily sea-surface velocity and save them as individual figures

In [5]:
%%time

session = cc.database.create_session()

def fancy_plot(ax):
    ax.gridlines(color='grey', linewidth=1, alpha=1, # dots as grid lines
                  xlocs=range(-180, 180, 60), # longitude grid lines
                  ylocs= np.linspace(-45, -90, num=4)) # latitude grid lines
    # ax.coastlines(); # add coast lines
    theta = np.linspace(0, 2*np.pi, 100); center, radius = [0.5, 0.5], .5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)
    
    # add labels manually
    xlab =    [   .99,      0,    1.01,    -.01,     .5,     .55,    .55,    .55] # x-position of labels
    ylab =    [   .75,    .75,     .24,     .24,   -.05,    .725,    .85,   .605] # y-position of labels
    txt_lab = ['60°E', '60°W', '120°E', '120°W', '180°',  '60°S', '45°S', '75°S'] # label text

    # loop through the 7 labels and surround with white space for higher visibility
    for l in range(len(txt_lab)):
        ax1.text(xlab[l], ylab[l], txt_lab[l], horizontalalignment='center', transform=ax1.transAxes, 
                fontsize=16).set_path_effects([PathEffects.withStroke(linewidth=2, foreground='w')]) 
        
# ----------------------------------------------------------------------------------------------------------- #
depth     = [    17]  # 48.98 m depth, subsurface to avoid flickering from high-frequency surface variability #
sel_lat   = [0, 940]  # 81.09°S - 29.15°S                                                                     #
# ----------------------------------------------------------------------------------------------------------- #
for t in range(10):#range(365): # loop through the time dimension, creating a frame for each daily output field
    filename = frame_directory + '10m_speed_frame_' + str('%03d' % (t,))+'.png' # name of frame to save as .png file

    for f in range(2): # loop through the two variables, u and v to calculate the speed (speed = u^2+v^2)
        variables = ['u', 'v']
        field = cc.querying.getvar(expt='01deg_jra55v140_iaf', variable=variables[f], 
                                   session=session, frequency='1 daily',
                                   attrs={'cell_methods': 'time: mean'},
                                   start_time='2012-01-01 00:00:00', 
                                   end_time='2012-12-31 00:00:00', 
                                   chunks = {'yu_ocean': '200MB', 'xu_ocean': '200MB'})[t, depth[0], sel_lat[0]:sel_lat[1], :]

        if f == 0: u = field # zonal velocity
        if f == 1: v = field # meridional velocity

    speed = (u**2 + v**2).load() # load 2D wind speed magnitude field into memory
    
    if os.path.isfile(filename) == True: # skip iteration if final .png file already exists
        print('Frame for '+str(speed.time)[36:46] + ' already done')
        continue

    # initialise figure
    fig = plt.figure(figsize=(8, 8), tight_layout=True, facecolor='w', edgecolor='k')
    gs = gridspec.GridSpec(1, 1)

    ax1 = plt.subplot(gs[0, 0], projection=ccrs.SouthPolarStereo(central_longitude=0))
    ax1.set_extent([-180, 180, -90, -30], crs=ccrs.PlateCarree()) # extent of plot

    blue_marble = plt.imread('/g/data/ik11/grids/BlueMarble.tiff')
    blue_marble_extent = (-180, 180, -90, 90) # extent of the land surface figure (same as above)
    
    # Add pretty land using the COSIMA recipe: https://cosima-recipes.readthedocs.io/en/latest/DocumentedExamples/Bathymetry.html

    # ---------------------------------------------------------------------------------------------------------------- #
    ax1.imshow(blue_marble, extent=blue_marble_extent, transform=ccrs.PlateCarree(), origin='upper')
    p1 = speed.plot.contourf(ax=ax1, levels=np.linspace(-0, 0.8, 21), cmap='Blues_r',
                             add_colorbar=False, extend='max', transform=ccrs.PlateCarree())

    # use function to add info (labels, land, etc), add title with date
    fancy_plot(ax1); plt.title('ACCESS-OM2-01, 50 m subsurface speed, ' + str(speed.time)[36:46]+'\n', fontsize=16)
    # ---------------------------------------------------------------------------------------------------------------- #

    # add colorbar
    cax = fig.add_axes([0.312, 0, 0.4, 0.012]) # position: [x0, y0, width, height] centered colour bar
    cb = plt.colorbar(p1, cax = cax, shrink=.5, orientation='horizontal') 
    cb.set_label(label='(m s$^{-1}$)', size=16) # colour bar label
    cb.ax.tick_params(labelsize=16)
    tick_locator = ticker.MaxNLocator(nbins=5) # five ticks
    cb.locator = tick_locator
    cb.update_ticks()

    # --- saving as 300 dpi .PNG image in specified folder ----------------------------------------------- #
    plt.savefig(filename, dpi=300, facecolor='w', edgecolor='w', orientation='landscape', format=None,     #
                transparent=False, bbox_inches='tight', pad_inches=0.1, metadata=None)                     #
    # --- end of script ---------------------------------------------------------------------------------- # 
    print('Frame for '+str(speed.time)[36:46] + ' done')
    if t != 1: plt.close(fig) # close figure if it's not the first one.
print('-------------------------') 
# Wall time: 2.33 s for just the frame of the figure
# Wall time: 1min 47s for one frame

MemoryError: Task '_preprocess-37faa17d-ec1d-457d-9d1c-9d03422df321' has 81.47 GiB worth of input dependencies, but worker tcp://127.0.0.1:41471 has memory_limit set to 23.57 GiB.

### Creat animation from the individually saved frames
- Warning if file already exists

In [6]:
%%time

from subprocess import check_call

check_call('ffmpeg -framerate ' + str(frame_rate) + 
           ' -pattern_type glob -i "'+
           frame_directory + '/10m_speed_frame_*.png" -vf scale=-2:' + str(resolution) + ',setsar=1 '+
           frame_directory + movie_name, shell=True)

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-7)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716145014501/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716145014501/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716145014501/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716145014501/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716145014501/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libo

CalledProcessError: Command 'ffmpeg -framerate 30 -pattern_type glob -i "./movie_frames//10m_speed_frame_*.png" -vf scale=-2:1080,setsar=1 ./movie_frames/test.mp4' returned non-zero exit status 234.

### Play the video
- this works when using Google Chrome or Microsoft Edge
- this does not work when using Mozilla Firefox

In [7]:
%%time

from IPython.display import Video
Video(frame_directory + movie_name, embed=True, height=500)

CPU times: user 0 ns, sys: 793 μs, total: 793 μs
Wall time: 564 μs
